In [0]:
from pyspark.sql.functions import *
import pyspark.sql.types as T
from pyspark.sql import Window

In [0]:
data = spark.read.parquet("/mnt/datalake/tmp/amani/data_node_way/way_2022")
data=data.orderBy("dateentry")

In [0]:
windows = Window.partitionBy("rideid").orderBy(col("dateentry").asc())
df_lag = (
          data.withColumn('lag_wayid', lag('id_way').over(windows))
              .withColumn('lead_wayid', lead('id_way').over(windows))
             
          
         )

In [0]:
df_lag=df_lag.withColumn("id_way2",when(df_lag.node_intersection==True,df_lag.lag_wayid).otherwise(df_lag.id_way))

In [0]:
df_lag=df_lag.withColumn("id_way3",when(col("id_way2").isNull(),df_lag.lead_wayid).otherwise(df_lag.id_way2))

In [0]:
df_lag.select("deviceid","rideId","dateentry").count()

In [0]:
#Drop duplicate rows
df_lag =df_lag.dropDuplicates(["deviceid","rideId","dateentry"])

In [0]:
df_count=df_lag.groupby(col("nodeId")).agg(count_distinct("id_way3").alias("count_way"))

In [0]:
#grouper par chemin en gardant la liste de node a fin d'ajouter ID 
df_way=df_lag.groupby("rideId","id_way3").agg(collect_set('nodeId').alias("list_nodes"))

In [0]:
df_way=df_way.withColumn("tronçon_physique", monotonically_increasing_id())
df_way=df_way.withColumn("tronçon_physique2", concat(col("id_way3"),col("tronçon_physique"))) 

In [0]:
#jointure final
data_tronçons = df_lag.join(df_way, ["rideId","id_way3"],"left")

In [0]:
data_tronçons.filter(col("rideId")=="01b04691-495b-4559-8430-8a4066d08acc_018317f27d811de5#00274356").orderBy(col("dateentry").asc()).display()

In [0]:
data_tronçons.select(medain)